
# Trader Behavior Insights — Junior Data Scientist Assignment

**Candidate:** K. Rajasekhar

**Objective:** Analyze relationship between trader performance (Hyperliquid trades) and Bitcoin market sentiment (Fear/Greed index). This Colab notebook contains data loading, cleaning, aggregation, merging with sentiment, EDA, visualizations, and key insights.

**Files provided:** `historical_data.csv`, `fear_greed_index.csv` (placed in `/content` when running in Colab) or use Google Drive links.

---


In [ ]:

# Setup - imports and helper functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Paths (adjust if using Colab with Drive)
DATA_DIR = Path('/content')  # replace with your drive mount path if needed
HIST = DATA_DIR/'historical_data.csv'
FG = DATA_DIR/'fear_greed_index.csv'


In [ ]:

# Load small samples to inspect
hist = pd.read_csv(HIST, low_memory=False, nrows=1000)
fg = pd.read_csv(FG, low_memory=False, nrows=1000)
print('Historical sample columns:', hist.columns.tolist())
print('Fear-Greed columns:', fg.columns.tolist())
hist.head()


In [ ]:

# Full processing (chunked) - produces daily_aggregates.csv and merged_daily_sentiment.csv
def process_historical(infile, out_daily='daily_aggregates.csv', chunksize=150000):
    cols = None
    agg_frames = []
    for chunk in pd.read_csv(infile, chunksize=chunksize, low_memory=False):
        # normalize columns
        chunk.columns = [c.strip() for c in chunk.columns]
        # detect time column
        time_col = None
        for c in chunk.columns:
            if 'time' in c.lower():
                time_col = c
                break
        if time_col is None:
            raise ValueError('No time column found in historical data.')
        chunk[time_col] = pd.to_datetime(chunk[time_col], errors='coerce')
        chunk['date'] = chunk[time_col].dt.date
        # numeric conversions
        for col in ['Closed PnL','Closed PnL ','ClosedPnL','closed_pnl','closed pnl','Closed PnL USD']:
            if col in chunk.columns:
                chunk['closed_pnl'] = pd.to_numeric(chunk[col].astype(str).str.replace(',',''), errors='coerce')
                break
        for col in ['Size USD','Size_USD','size_usd','size usd']:
            if col in chunk.columns:
                chunk['size_usd'] = pd.to_numeric(chunk[col].astype(str).str.replace(',',''), errors='coerce')
                break
        for col in ['Fee','fee']:
            if col in chunk.columns:
                chunk['fee'] = pd.to_numeric(chunk[col].astype(str).str.replace(',',''), errors='coerce')
                break
        daily = chunk.groupby('date').agg(
            trades_count = ('closed_pnl','count'),
            pnl_sum = ('closed_pnl','sum'),
            pnl_mean = ('closed_pnl','mean'),
            pnl_median = ('closed_pnl','median'),
            win_count = ('closed_pnl', lambda s: (s>0).sum()),
            avg_size_usd = ('size_usd','mean') if 'size_usd' in chunk.columns else ('closed_pnl','mean'),
            avg_fee = ('fee','mean') if 'fee' in chunk.columns else ('closed_pnl','mean')
        ).reset_index()
        agg_frames.append(daily)
    agg_all = pd.concat(agg_frames, ignore_index=True)
    agg_final = agg_all.groupby('date').agg(
        trades_count = ('trades_count','sum'),
        pnl_sum = ('pnl_sum','sum'),
        pnl_mean = ('pnl_mean','mean'),
        pnl_median = ('pnl_median','mean'),
        win_count = ('win_count','sum'),
        avg_size_usd = ('avg_size_usd','mean'),
        avg_fee = ('avg_fee','mean')
    ).reset_index()
    agg_final['win_rate'] = agg_final['win_count'] / agg_final['trades_count']
    agg_final.to_csv(out_daily, index=False)
    return agg_final

def merge_with_sentiment(daily_csv, fg_csv, out_merged='merged_daily_sentiment.csv'):
    df_daily = pd.read_csv(daily_csv, parse_dates=['date'])
    fg = pd.read_csv(fg_csv, parse_dates=['date'])
    df_daily['date'] = pd.to_datetime(df_daily['date']).dt.date
    fg['date'] = pd.to_datetime(fg['date']).dt.date
    merged = pd.merge(df_daily, fg[['date','classification']], on='date', how='left')
    merged.to_csv(out_merged, index=False)
    return merged



## Suggested Analyses

- Compare **avg daily PnL** and **win rate** between `Fear` vs `Greed` days.
- Visualizations to include in `outputs/`:
  - Boxplot of daily `pnl_sum` by `classification`
  - Time series of `pnl_sum` with sentiment markers
  - Scatter: avg_leverage vs pnl_sum (if leverage exists)
  - Top symbols profitability by sentiment

## Deliverables
- `notebook_1.ipynb` (this notebook)
- `csv_files/daily_aggregates.csv`
- `csv_files/merged_daily_sentiment.csv`
- `outputs/*.png`
- `ds_report.pdf`
